In [1]:
import os
import pandas as pd
import numpy as np
import h5py

In [2]:
source_folder = '/home/jupyter/hw-text/'

In [3]:
all_csv_files = list(filter(lambda x: '.csv' in x, os.listdir(source_folder)))

In [4]:
data_frames = []
for f in all_csv_files:
    file_with_path = os.path.join(source_folder, f)
    img_folder = f.split('.')[0]
    img_folder = img_folder.split('_')
    img_folder[0] = 'data'
    img_folder = '_'.join(img_folder)
    img_folder = os.path.join(source_folder, img_folder)
    df = pd.read_csv(file_with_path)
    df = df.drop('Unnamed: 0', axis=1)
    df.filename = df.filename.apply(lambda x: os.path.join(img_folder, x))
    data_frames.append(df)

concat_data_frames = pd.concat(data_frames)

In [5]:
train, validate, test = np.split(concat_data_frames.sample(frac=1), [int(.7*len(concat_data_frames)), int(.9*len(concat_data_frames))])

In [6]:
len(train)/len(concat_data_frames), len(validate)/len(concat_data_frames), len(test)/len(concat_data_frames)

(0.7, 0.2, 0.1)

In [7]:
dt = {'train': train,
     'valid': validate,
     'test': test}

In [8]:
partitions = ['train', 'valid', 'test']
dataset = {}
for i in partitions:
    dataset[i] = {"dt": list(dt[i].filename.values), "gt": list(dt[i].text.values)}

In [9]:
from data.reader import Dataset

In [10]:
input_size = (1024, 128, 1)
model_dataset = Dataset('none', 'none')
model_dataset.dataset = dataset

In [11]:
model_dataset.preprocess_partitions(input_size=input_size)

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
h5py_file = os.path.join(source_folder, 'hw-text.hdf5')
for i in partitions:
    with h5py.File(h5py_file, "a") as hf:
        hf.create_dataset("{}/dt".format(i), data=model_dataset.dataset[i]['dt'], compression="gzip", compression_opts=9)
        hf.create_dataset("{}/gt".format(i), data=model_dataset.dataset[i]['gt'], compression="gzip", compression_opts=9)
        print("[OK] {} partition.".format(i))
print('done')